In [1]:
import wandb
wandb.login()

<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

Retry attempt failed:
Traceback (most recent call last):
  File "/home/quanhhh/miniconda3/envs/fedml-torch/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/home/quanhhh/miniconda3/envs/fedml-torch/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/quanhhh/miniconda3/envs/fedml-torch/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/quanhhh/miniconda3/envs/fedml-torch/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "/home/quanhhh/miniconda3/envs/fedml-torc

In [ ]:
import os
import yaml
import sys
import time
from copy import deepcopy
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
with open('/home/quanhhh/Documents/RUL_Transformer/bo.yml') as f:
  sweep_config = yaml.safe_load(f)

In [ ]:
#model.py

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dff, dropout):
        super().__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.dff = dff
        self.dropout = dropout
        self.encoderlayer = nn.TransformerEncoderLayer(
            d_model=self.d_model,
            nhead=self.nhead,
            dim_feedforward=self.dff,
            dropout=self.dropout,
            batch_first=True,
        )

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        out = self.encoderlayer(src)
        return out


class TransformerEncoder(nn.Module):
    def __init__(self, encoder_layer, num_layers):
        super().__init__()
        self.encoder_layer = encoder_layer
        self.num_layer = num_layers
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

    def forward(self, src):
        out = self.transformer_encoder(src)
        return out


class TransformerModel(nn.Module):
    def __init__(self, encoder, linear):
        super().__init__()
        self.encoder = encoder
        self.linear = linear

    def forward(self, src):
        out = F.relu(self.linear(self.encoder(src)))
        return out


def create_transformer(d_model, nhead, dff, num_layers, dropout, l_win):
    linear = nn.Sequential(
        nn.Flatten(), nn.Dropout(dropout), nn.Linear(d_model * l_win, 1)
    )
    model = TransformerModel(
        TransformerEncoder(
            TransformerEncoderLayer(d_model, nhead, dff, dropout), num_layers
        ),
        linear,
    )

    for p in model.parameters():
      if p.dim() > 1:
        nn.init.xavier_uniform_(p)
        
    return model


#dataloader.py
class TimeSeriesDataset(Dataset):
    def __init__(self, config, mode):
        super().__init__()
        self.config = config
        self.mode = mode
        self.load_dataset(config)

    def __getitem__(self, idx):
        if self.mode == 'train':
            data = self.data[idx, :, :]
            label = self.label[idx]
            return data, label
        else:
            data = self.data[idx, :, :]
            label = self.label[idx]
            return data, label

    def __len__(self):
        return self.data.shape[0]

    def load_dataset(self, config):
        if self.mode == 'train':
            train_df = pd.read_csv('/home/quanhhh/Documents/RUL_Transformer/preprocessed_data/train_004.csv')

            def gen_sequence(id_df, seq_length, seq_cols):
                data_array = id_df[seq_cols].values
                num_elements = data_array.shape[0]
                for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
                    yield data_array[start:stop, :]
            
            sensor_cols = [
                "s1",
                "s2",
                "s3",
                "s4",
                "s5",
                "s7",
                "s8",
                "s9",
                "s10",
                "s11",
                "s12",
                "s13",
                "s14",
                "s15",
                "s16",
                "s17",
                "s18",
                "s19",
                "s20",
                "s21",
            ]
            sequence_cols = ["setting1", "setting2", "setting3"]
            sequence_cols.extend(sensor_cols)
            # generator for the sequences
            seq_gen = (list(gen_sequence(train_df[train_df['id']==id], self.config['l_win'], sequence_cols)) 
                      for id in train_df['id'].unique())

            # generate sequences and convert to numpy array
            seq_array = np.concatenate(list(seq_gen)).astype(np.float32)

            # function to generate labels
            def gen_labels(id_df, seq_length, label):
                data_array = id_df[label].values
                num_elements = data_array.shape[0]
                return data_array[seq_length:num_elements, :]

            # generate labels
            label_gen = [gen_labels(train_df[train_df['id']==id], self.config['l_win'], ['RUL']) 
                        for id in train_df['id'].unique()]
            label_array = np.concatenate(label_gen).astype(np.float32)

            self.data = seq_array
            self.label = label_array

        else:
            test_df = pd.read_csv('/home/quanhhh/Documents/RUL_Transformer/preprocessed_data/test_004.csv')

            sensor_cols = [
                "s1",
                "s2",
                "s3",
                "s4",
                "s5",
                "s7",
                "s8",
                "s9",
                "s10",
                "s11",
                "s12",
                "s13",
                "s14",
                "s15",
                "s16",
                "s17",
                "s18",
                "s19",
                "s20",
                "s21",
            ]
            sequence_cols = ["setting1", "setting2", "setting3"]
            sequence_cols.extend(sensor_cols)

            seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-config['l_win']:] 
                                  for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= config['l_win']]

            seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)


            y_mask = [len(test_df[test_df['id']==id]) >= config['l_win'] for id in test_df['id'].unique()]

            label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
            label_array_test_last = label_array_test_last.reshape(label_array_test_last.shape[0],1).astype(np.float32)

            self.data = seq_array_test_last
            self.label = label_array_test_last




#trainer.py
class ModelTrainer():
    def __init__(self, model, train_data, criterion, optimizer, device, config):
        self.model = model
        self.train_data = train_data
        self.device = device
        self.config = config
        self.train_loss_list = list()
        self.min_loss = float('inf')
        self.best_model = None
        self.best_optimizer = None
        self.optimizer = optimizer
        self.criterion = criterion

    def train_epoch(self, epoch):
        train_loss = 0.0
        self.model.train()
        for x, rul in self.train_data:
            self.model.zero_grad()
            out = self.model(x.to(self.device).float())
            loss = torch.sqrt(self.criterion(out.float(), rul.to(self.device).float()))
            loss.backward()
            self.optimizer.step()
            train_loss += loss
            

        train_loss = train_loss / len(self.train_data)
        wandb.log({"train loss": train_loss})
        self.train_loss_list.append(train_loss)

        if train_loss < self.min_loss:
            self.min_loss = train_loss
            self.best_model = deepcopy(self.model.state_dict())
            self.best_optimizer = deepcopy(self.optimizer.state_dict())
            self.best_epoch_in_round = epoch

    def train(self):
        self.model.to(self.device)

        for epoch in range(1, self.config['n_epochs'] + 1):
            self.train_epoch(epoch)
            wandb.log({"epoch": epoch})


        self.config['train_loss_list'] = self.train_loss_list

    def update_config(self):
        return self.config

In [ ]:
#train.py
def training():
    with wandb.init(config=sweep_config):
        config = wandb.config

        torch.manual_seed(42)
        start = time.perf_counter()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        train_data = TimeSeriesDataset(config, mode='train')
        train_loader = DataLoader(train_data,
                                  batch_size=config['batch_size'],
                                  shuffle=True,
                                  num_workers=config['num_workers'])

        model = create_transformer(d_model=config['d_model'],
                                    nhead=config['n_head'],
                                    dff=config['dff'],
                                    num_layers=config['num_layers'],
                                    dropout=config['dropout'],
                                    l_win=config['l_win'])

        optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config['weight_decay'])
        criterion = nn.MSELoss()
        trainer = ModelTrainer(model, train_loader, criterion, optimizer, device, config)

        trainer.train()

        #inference.py

        test_data = TimeSeriesDataset(config, mode='test')
        test_loader = DataLoader(test_data,
                                    batch_size=1,
                                    shuffle=False,
                                    num_workers=config['num_workers'])

        model.to(device)
        test_loss = 0.0
        criterion = nn.MSELoss()
        test_loss_list = list()
        pred_list = list()
        with torch.no_grad():
            for x, rul in test_loader:
                out = model(x.to(device).float())
                loss = torch.sqrt(criterion(out.float(), rul.to(device).float()))
                test_loss += loss
                test_loss_list.append(loss)
                pred_list.append(out.float())

        test_loss_avg = test_loss / len(test_loader)
        truth_list = [rul.float().item() for x, rul in test_loader]
        config['truth_list'] = truth_list
        config['pred_list'] = pred_list
        config['test_loss_avg'] = test_loss_avg
        config['test_loss_list_per_id'] = test_loss_list
        wandb.log({"test_loss_avg": test_loss_avg})
        if test_loss_avg < 12.4:
            wandb.alert(
                title='longmeow come first with ey2demty',
                text=f'test_loss_avg {test_loss_avg} is below the theshold 12.4',
            )
            print('Alert triggered')    


In [ ]:
!date
wandb.agent('rul-project/6so07szz', training, count=100000)